# Data prep

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import json

import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
#Read meta data
root_path = '2020-03-13'
metadata_path = root_path+'/all_sources_metadata_2020-03-13.csv'
meta_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str, 
    'doi': str
})
meta_df.head()

,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text
0,c630ebcdf30652f0422c3ec12a00b50241dc9bd9,CZI,Angiotensin-converting enzyme 2 (ACE2) as a SA...,10.1007/s00134-020-05985-9,NaN,32125455,cc-by-nc,NaN,2020,"Zhang, Haibo; Penninger, Josef M.; Li, Yimin; ...",Intensive Care Med,2002765492,#3252,True
1,53eccda7977a31e3d0f565c884da036b1e85438e,CZI,Comparative genetic analysis of the novel coro...,10.1038/s41421-020-0147-1,NaN,NaN,cc-by,NaN,2020,"Cao, Yanan; Li, Lin; Feng, Zhimin; Wan, Shengq...",Cell Discovery,3003430844,#1861,True
2,210a892deb1c61577f6fba58505fd65356ce6636,CZI,Incubation Period and Other Epidemiological Ch...,10.3390/jcm9020538,NaN,NaN,cc-by,The geographic spread of 2019 novel coronaviru...,2020,"Linton, M. Natalie; Kobayashi, Tetsuro; Yang, ...",Journal of Clinical Medicine,3006065484,#1043,True
3,e3b40cc8e0e137c416b4a2273a4dca94ae8178cc,CZI,Characteristics of and Public Health Responses...,10.3390/jcm9020575,NaN,32093211,cc-by,"In December 2019, cases of unidentified pneumo...",2020,"Deng, Sheng-Qun; Peng, Hong-Juan",J Clin Med,177663115,#1999,True
4,92c2c9839304b4f2bc1276d41b1aa885d8b364fd,CZI,Imaging changes in severe COVID-19 pneumonia,10.1007/s00134-020-05976-w,NaN,32125453,cc-by-nc,NaN,2020,"Zhang, Wei",Intensive Care Med,3006643024,#3242,False


In [3]:
meta_df["doi_modified"] = meta_df["doi"].str.replace('/','%2F')
meta_df["URL"] = "https://www.google.com/search?q=doi+"+ meta_df["doi_modified"]

In [4]:
all_json = glob.glob(root_path+"/**/*.json", recursive=True)
len(all_json)

13202

In [5]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return (f'{self.paper_id}: {self.abstract[:200]}... {self.body_text[:200]}...')
    
first_row = FileReader(all_json[0])
print(first_row)

0015023cc06b5362d332b3baf348d11567ca2fbb: word count: 194 22 Text word count: 5168 23 24 25 author/funder. All rights reserved. No reuse allowed without permission. Abstract 27 The positive stranded RNA genomes of picornaviruses comprise a si... VP3, and VP0 (which is further processed to VP2 and VP4 during virus assembly) (6). The P2 64 and P3 regions encode the non-structural proteins 2B and 2C and 3A, 3B (1-3) (VPg), 3C pro and 4 structura...


In [6]:
def get_breaks(content, length):
    data = ""
    words = content.split(' ')
    total_chars = 0

    # add break every length characters
    for i in range(len(words)):
        total_chars += len(words[i])
        if total_chars > length:
            data = data + "<br>" + words[i]
            total_chars = 0
        else:
            data = data + " " + words[i]
    return data

In [7]:
from ast import literal_eval

dict_ = {'paper_id': [], 'abstract': [], 'body_text': [], 'authors': [], 'title': [], 'journal': [], 'abstract_summary': []}
for idx, entry in enumerate(all_json):
    if idx % (len(all_json) // 10) == 0:
        print(f'Processing index: {idx} of {len(all_json)}')
    content = FileReader(entry)
    dict_['paper_id'].append(content.paper_id)
    dict_['abstract'].append(content.abstract)
    dict_['body_text'].append(content.body_text)
      
    # also create a column for the summary of abstract to be used in a plot
    if len(content.abstract) == 0: 
        # no abstract provided
        dict_['abstract_summary'].append("Not provided.")
    elif len(content.abstract.split(' ')) > 100:
        # abstract provided is too long for plot, take first 300 words append with ...
        info = content.abstract.split(' ')[:100]
        summary = get_breaks(' '.join(info), 40)
        dict_['abstract_summary'].append(summary + "...")
    else:
        # abstract is short enough
        summary = get_breaks(content.abstract, 40)
        dict_['abstract_summary'].append(summary)
        
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    try:
        # if more than one author
        authors = literal_eval(meta_data['authors'].values[0])
        if len(authors) > 2:
            # more than 2 authors, may be problem when plotting, so take first 2 append with ...
            dict_['authors'].append(". ".join(authors[:2]) + "...")
        else:
            # authors will fit in plot
            dict_['authors'].append(". ".join(authors))
    except Exception as e:
        # if only one author - or Null valie
        dict_['authors'].append(meta_data['authors'].values[0])
    
    # add the title information, add breaks when needed
    try:
        title = get_breaks(meta_data['title'].values[0], 40)
        dict_['title'].append(title)
    # if title was not provided
    except Exception as e:
        dict_['title'].append(meta_data['title'].values[0])
    
    # add the journal information
    dict_['journal'].append(meta_data['journal'].values[0])
    
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text', 'authors', 'title', 'journal', 'abstract_summary'])
df_covid.head()

Processing index: 0 of 13202
Processing index: 1320 of 13202
Processing index: 2640 of 13202
Processing index: 3960 of 13202
Processing index: 5280 of 13202
Processing index: 6600 of 13202
Processing index: 7920 of 13202
Processing index: 9240 of 13202
Processing index: 10560 of 13202
Processing index: 11880 of 13202
Processing index: 13200 of 13202


,paper_id,abstract,body_text,authors,title,journal,abstract_summary
0,0015023cc06b5362d332b3baf348d11567ca2fbb,word count: 194 22 Text word count: 5168 23 24...,"VP3, and VP0 (which is further processed to VP...","Ward, J. C. J.; Lasecka-Dykes, L.; Neil, C.; A...",The RNA pseudoknots in foot-and-mouth disease...,NaN,word count: 194 22 Text word count: 5168 23 2...
1,004f0f8bb66cf446678dc13cf2701feec4f36d76,,The 2019-nCoV epidemic has spread across China...,NaN,NaN,NaN,Not provided.
2,00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,Infectious bronchitis (IB) causes significant ...,"Infectious bronchitis (IB), which is caused by...","Butt, S. L.; Erwood, E. C.; Zhang, J.; Sellers...","Real-time, MinION-based, amplicon<br>sequenci...",NaN,Infectious bronchitis (IB) causes<br>signific...
3,013d9d1cba8a54d5d3718c229b812d7cf91b6c89,Background: A novel coronavirus (2019-nCoV) em...,"In December 2019, a cluster of patients with p...",NaN,NaN,NaN,Background: A novel coronavirus (2019-nCoV)<b...
4,01d162d7fae6aaba8e6e60e563ef4c2fca7b0e18,Faced with the current large-scale public heal...,The sudden outbreak of the new coronavirus (SA...,NaN,NaN,NaN,Faced with the current large-scale public<br>...


In [9]:
# Filter top 5 journals only

df_covid = df_covid[df_covid["journal"].isin(list(df_covid.groupby("journal")["paper_id"].agg("count").sort_values(ascending=False)[0:5].index))]
df_covid=df_covid.dropna()
df_covid.shape

# Topic Model

In [41]:
import NewsTrends
import importlib
importlib.reload(NewsTrends)
import locale
import matplotlib.pyplot as plt
plt.style.use('ggplot')

%matplotlib inline

In [42]:
t = NewsTrends.topicModel(df_covid,
                          key_idx=list(df_covid.columns).index("paper_id"),
                          text_idx=list(df_covid.columns).index("body_text"),
                          lang="en_core_web_sm",random_state = 1,
                          bigram = False)

In [43]:
corpus = t.tokenize_docs(rmv_tokens=["'s","=","datum","ma","virus","viral","p","word","count", "r","t","ifn","+","c","ml"])

3825 done ]


In [44]:
corpus.head()

,paper_id,tokenized_text
0,000b7d1517ceebb34e1e3e817695b6de03e2fa78,"[figure, s1, phylogeny, sequence, belong, umrv..."
1,0072159e1ebecc889e9bcabb58bb45c47e18a403,"[introduction, ebola, ebov, marburg, marv, vir..."
2,007bf75961da42a7e0cc8e2855e5c208a5ec65c1,"[initiate, infection, bind, appropriate, host,..."
3,0080d3bd9fb92e022c27715c2d1249042aa998b8,"[dengue, denv, member, flaviviridae, family, d..."
4,0089aa4b17549b9774f13a9e2e12a84fc827d60b,"[medium, chain, acyl, coa, dehydrogenase, defi..."


In [45]:
corpus = t.get_bow(below = 0.05,above = 0.9)

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(174224 unique tokens: ['-table', 'accession', 'aquaparamyxovirus', 'b', 'bat']...) from 3826 documents (total 8942355 corpus positions)
INFO:gensim.corpora.dictionary:discarding 171395 tokens: [('-table', 3), ('aquaparamyxovirus', 5), ('burn', 79), ('cytochrome', 106), ('ef646380', 2), ('gtr', 46), ('iteration', 113), ('malagasy', 7), ('morbillivirus', 36), ('mrbayes', 33)]...
INFO:gensim.corpora.dictionary:keeping 2829 tokens which were in no less than 191 and no more than 3443 (=90.0%) documents
INFO:gensim.corpora.dictionary:resulting dictionary: Dictionary(2829 unique tokens: ['accession', 'b', 'bat', 'belong', 'calculate']...)
INFO:gensim.corpora.mmcorpus:storing corpus in Matrix Market format to Corpus.mm
INFO:gensim.matutils:saving sparse matrix to Corpus.mm
INFO:gensim.matutils:PROGRESS: saving document #0
INFO:gensim.matutils:PROGRESS: saving doc

In [46]:
#k,results = t.LDA_tune_k(max_k=20,iterations=800)

In [47]:
#plt.plot(k,results)

In [48]:
#Train topic model
LDA = t.fit_LDA(6,iterations=2000)

INFO:lda:n_documents: 3826
INFO:lda:vocab_size: 2829
INFO:lda:n_words: 6708490
INFO:lda:n_topics: 6
INFO:lda:n_iter: 2000
INFO:lda:<0> log likelihood: -60755753
INFO:lda:<10> log likelihood: -53580345
INFO:lda:<20> log likelihood: -50891646
INFO:lda:<30> log likelihood: -50711963
INFO:lda:<40> log likelihood: -50660035
INFO:lda:<50> log likelihood: -50638882
INFO:lda:<60> log likelihood: -50622800
INFO:lda:<70> log likelihood: -50606381
INFO:lda:<80> log likelihood: -50600163
INFO:lda:<90> log likelihood: -50587447
INFO:lda:<100> log likelihood: -50580137
INFO:lda:<110> log likelihood: -50572997
INFO:lda:<120> log likelihood: -50567478
INFO:lda:<130> log likelihood: -50562854
INFO:lda:<140> log likelihood: -50562824
INFO:lda:<150> log likelihood: -50558855
INFO:lda:<160> log likelihood: -50559001
INFO:lda:<170> log likelihood: -50557247
INFO:lda:<180> log likelihood: -50558694
INFO:lda:<190> log likelihood: -50556381
INFO:lda:<200> log likelihood: -50558752
INFO:lda:<210> log likelihoo

INFO:lda:<1950> log likelihood: -50521494
INFO:lda:<1960> log likelihood: -50522304
INFO:lda:<1970> log likelihood: -50514743
INFO:lda:<1980> log likelihood: -50515775
INFO:lda:<1990> log likelihood: -50519822
INFO:lda:<1999> log likelihood: -50522636


In [49]:
#Inspect topic results
topics = t.inspect_topics(n_top_words=15)

res = ''
for i, topic_words in enumerate(topics):
    res += '* **Topic {}:** {}\n'.format(i, ' '.join(topic_words))
NewsTrends.MD(res)

'* **Topic 0:** patient infection respiratory cov case sars influenza sample clinical test age mers child disease\n* **Topic 1:** protein cell rna bind fig membrane activity replication structure figure domain interaction residue site\n* **Topic 2:** sequence gene sample strain genome bat rna pcr analysis specie human table primer identify\n* **Topic 3:** disease model case number health time population rate outbreak individual transmission estimate risk contact\n* **Topic 4:** cell infection mouse expression response ifn gene level infect induce immune increase fig t\n* **Topic 5:** cell antibody assay fig control anti h vaccine concentration culture test time sample protein\n'

In [50]:
topics = [["patient, infection, respiratory, age, child"],
          ["protein, cell, rna, bind, membrane, structure"],
          ["pcr, analysis, identify, sequence, genome"],
          ["model, rate, outbreak, transmission, risk"],
          ["mouse, response, infect, immune"],
          ["cell, antibody, vaccine, assay"]]
len(topics)

6

In [55]:
doctopics = t.get_doc_topic(topics)
graph = t.generate_network(node_attr=[ "topic",u'authors', u'journal',"abstract"],
                  node_label="title", similarity_cutoff = 0.98,
                 similarity_measure = "correlation")

In [56]:
import networkx as nx

graph.remove_nodes_from(list(nx.isolates(graph)))
nx.write_gml(graph, 'docs.gml')

# COVID-19 research exploration
By using intelligent text processing and learning directly from the textual context, the following analysis finds meaningful connections and relationships in a large corpus of unstructured data. Following I invite you to interactively explore the large body of research around COVID-19. This analysis aims to help to better understand the disease.

# Data summary

In [22]:
analysiscode="COVID3"

In [34]:
import pprint
print("The cleaned data set with only the top 5 journals contains {} research papers from {} sets of authors.".format(t.df.shape[0],t.df["authors"].nunique()))
print()
print("The top journals in the data set are:")
print()
for a, b in enumerate(t.df.groupby("journal")["paper_id"].agg("count").sort_values(ascending=False)[0:5].keys(), 1):
    print('{} {}'.format(a, b))
    
print()
print("The data is sourced from https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge")

The cleaned data set with only the top 5 journals contains 3826 research papers from 3575 sets of authors.

The top journals in the data set are:

1 PLoS One
2 Emerg Infect Dis
3 Viruses
4 Sci Rep
5 PLoS Pathog

The data is sourced from https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge


# Document Analysis

*Note: If the following plots are not displayed nicely or missing the legend on the right-hand side, hit refresh and wait until fully loaded.*

In [61]:
import os
from shutil import copyfile

# Move templates
source = "01_Vorlage/"
dst = "graphics2/network/"

for f in ["index.html","css/style.css","js/main.js","js/sigma/sigma.parseJson.js"]:
    copyfile(source+f, dst+f)


### Research is mainly focused on the 1) epidemiology and 2) molecular biology of the virus ([How to read the network](http://nbviewer.jupyter.org/github/bockjo/Udacity_portfolio/blob/master/networks_how_to.png)).

In [64]:
from IPython.display import IFrame
IFrame("graphics2/network/index.html", width="100%", height=650)

# Plotting code

In [25]:
import plotly.io as pio
import warnings
pio.templates.default = 'simple_white'
warnings.filterwarnings('ignore')

In [26]:
#Plotly
import numpy as np
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
from ipywidgets import widgets
from chart_studio.widgets import GraphWidget

init_notebook_mode(connected=True)

# generate a function to handle changes in the widget
# add top_n functionality!!!
def timeline(t,y,date_col,colorby,aggregate="nunique",timeinterval=None):

    plot_dat = t.copy()
    plot_dat[date_col] = pd.to_datetime(plot_dat[date_col]).dt.week
        
    plot_dat = plot_dat.groupby([date_col,colorby])[y].agg(aggregate)
    dat = plot_dat.reset_index()
    
    dataset = []
    for gr in dat[colorby].unique():
        dataset.append(go.Bar(
        x=dat[dat[colorby]==gr][date_col],
        y=dat[dat[colorby]==gr][y],
        name=gr
    ))

    layout = go.Layout(
        autosize=True,
        barmode='stack',
        showlegend = True,
        legend=dict(orientation="h",x=0.1,y=1.5),
        yaxis = dict(title = "Publish-Count"),
        xaxis = dict(title="Week number")
    )
    
    
    fig = go.Figure(data=dataset, layout=layout)
    return iplot(fig, filename=analysiscode+'stacked-bar')

# generate a function to handle changes in the widget
def hist(t,x,colorby="topic_terms",logx = False):

    plot_dat = t.copy()
    
    if logx:
        plot_dat[x]=np.log(plot_dat[x])
        #typ = 'log'
    #else:
        #typ="normal"
    data = []
    for gr in plot_dat[colorby].unique():
        data.append(go.Histogram(
            x=plot_dat.loc[plot_dat[colorby]==gr,x],
            name=gr
            
    ))

    layout = go.Layout(barmode='stack',legend=dict(orientation="h",x=0.1,y=1.5),
                       autosize=True,
                #width=800,
                #height=550,
                       xaxis=dict(title=x,type="normal",autorange=True,
                                 exponentformat= "e" if logx else "none"),
                      yaxis = dict(title="count"))
    fig = go.Figure(data=data, layout=layout)

    
    return iplot(fig, filename=analysiscode+'stacked histogram')

# add top_n functionality!!!
def scatter(t,x,y,groupby,colorby = None,sizeby = "fixed", aggregate = {"x":"sum","y":"sum","colorby":"sum"},
            axistype = {"x":"linear","y":"linear"},
            xtitle="", ytitle=""):
    
    plot_dat = t.copy()
    if sizeby=="fixed":
        s=15
    if colorby == None:
        colorby = groupby
        
    if plot_dat[colorby].dtype == np.object:
        plot_datx = plot_dat.groupby([colorby,groupby])[x].agg(aggregate["x"])
        plot_daty = plot_dat.groupby([colorby,groupby])[y].agg(aggregate["y"])
        
        dat = []
        for cat in plot_dat[colorby].unique():
            
            dat.append({"x":plot_datx[cat].values,
                       "y":plot_daty[cat].values,
                        "text":plot_daty[cat].index,
                       "name":cat,
                       "mode":"markers",
                       "marker":dict(size=s)})
        fig = {
            'data': dat,
            'layout': {"legend":dict(orientation="h",x=0.1,y=1.5),"showlegend": True,
                'xaxis': {'title': xtitle,"type":axistype["x"],"exponentformat": "e" if axistype["x"]=="log" else "none"},
                'yaxis': {'title': ytitle,"type":axistype["y"],"exponentformat": "e" if axistype["y"]=="log" else "none"},
                "autosize":True
            }
        }
    else:
        plot_datx = plot_dat.groupby(groupby)[x].agg(aggregate["x"])
        plot_daty = plot_dat.groupby(groupby)[y].agg(aggregate["y"])
        plot_datcol = plot_dat.groupby(groupby)[colorby].agg(aggregate["colorby"])
        
        trace1 = go.Scatter(
            y = plot_daty.values,
            x = plot_datx.values,
            mode='markers',
            marker=dict(
                size=s,
                color = plot_datcol.values, #set color equal to a variable
                colorscale='Viridis',
                showscale=True
            )
        )
        layout = dict(xaxis=dic(title=xtitle,type=axistype["x"],exponentformat= "e" if axistype["x"]=="log" else "none"),
                      yaxis=dic(title=ytitle,type=axistype["y"],exponentformat= "e" if axistype["y"]=="log" else "none"))
        fig = go.Figure(data=[trace1],layout = layout)
        
    return iplot(fig, filename=analysiscode+'bar')

# add top_n functionality!!!
def bar_continuous(t):
    plot_dat = t.copy()
    
    plot_dat = plot_dat.groupby(groupby)[x].agg(aggregate)
    data =[
        go.Bar(
        y=plot_dat.index.values,
        x=plot_dat.values,
        orientation = 'h')
    ]
    fig = go.Figure(data=data)
    
    return iplot(fig, filename='bar_cont')
        
def bar(t,x,groupby,xtitle="",colorby = "topic_terms", aggregate = "sum"):
    plot_dat = t.copy()
    
    data = []
    plot_dat = plot_dat.groupby([colorby,groupby])[x].agg(aggregate)
    for gr in plot_dat.index.levels[0]: 
        data.append(go.Bar(
        y=plot_dat[gr].index.values,
        x=plot_dat[gr].values,
        name=gr,
        orientation = 'h'
    ))

    layout = go.Layout(
        autosize=False,
    width=1000,
    height=550,
        barmode='stack',
        showlegend = True,
        xaxis = dict(title=xtitle),#,position=-1.
        yaxis = dict(),
        legend=dict(orientation="v",x=1,y=1.2),
        margin = go.Margin(l=350,r=10)
    )

    fig = go.Figure(data=data, layout=layout)
    
    return iplot(fig, filename=analysiscode+'bar_cat')

<IPython.core.display.Javascript object>

# Exploratory analysis

### The top 5 journals cover most of the topics around the coronavirus, but PLoS One gives the most balanced and comprehensive view.

In [27]:
topjour = list(t.df.groupby("journal")["paper_id"].agg("count").sort_values(ascending=False)[0:5].index)

In [28]:
bar(t.df[t.df.journal.isin(topjour)],"paper_id","journal",xtitle="Publish Count",colorby = "topic", aggregate = "nunique")

### The top 10 authors tend to focus on their field of expertise, however Oved et al. cover a wider range of topics.

In [29]:
topau = list(t.df.groupby("authors")["paper_id"].agg("count").sort_values(ascending=False)[0:10].index)

In [30]:
bar(t.df[t.df.authors.isin(topau)],"paper_id","authors",xtitle="Publish Count",colorby = "topic", aggregate = "nunique")